<a href="https://colab.research.google.com/github/the-mir/CNN/blob/main/flowers_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
DIRECTORY = r"/content/drive/MyDrive/flowers"
CATEGORIES = ['tulip', 'peonies','orchids','lilies','hydrangeas','hibiscus','gardenias','garden_roses','daisies','bougainvillea']

In [52]:
data = []

for category in CATEGORIES:
    folder = os.path.join(DIRECTORY, category)
    label = CATEGORIES.index(category)

    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr, (100, 100))
        data.append([img_arr, label])

In [53]:
data

[[array([[[  3,  20,  17],
          [  0,  17,  15],
          [  2,  16,  15],
          ...,
          [  2,  22,  22],
          [  3,  20,  16],
          [  1,  14,  12]],
  
         [[  3,  19,  18],
          [  3,  19,  18],
          [  2,  19,  16],
          ...,
          [  1,  22,  20],
          [  3,  17,  15],
          [  2,  15,  13]],
  
         [[  2,  19,  16],
          [  3,  19,  18],
          [  1,  18,  15],
          ...,
          [  1,  20,  17],
          [  2,  16,  14],
          [  1,  14,  12]],
  
         ...,
  
         [[ 23, 105,  77],
          [ 26, 102,  69],
          [ 21, 105,  73],
          ...,
          [  2,   7,   5],
          [  3,   5,   5],
          [  0,   5,   3]],
  
         [[ 24,  98,  72],
          [ 11,  94,  71],
          [ 19,  94,  66],
          ...,
          [  0,   5,   3],
          [  1,   6,   5],
          [  0,   5,   4]],
  
         [[ 16,  93,  59],
          [ 11,  91,  57],
          [  8,  92,  59

In [54]:
random.shuffle(data)

In [55]:
# Separate the features (X) and labels (Y) from the data list
X = np.array([entry[0] for entry in data])
Y = np.array([entry[1] for entry in data])

In [56]:
# Normalize the image data
X = X / 255.0

In [57]:
Y = to_categorical(Y, len(CATEGORIES))

In [58]:
# Create a CNN model
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(len(CATEGORIES), activation='sigmoid'))

In [59]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X, Y, epochs=15, validation_split=0.1)

Epoch 1/15
21/21 [==============================] - 19s 864ms/step - loss: 2.2680 - accuracy: 0.1342 - val_loss: 2.1181 - val_accuracy: 0.2568
Epoch 2/15
 7/21 [=========>....................] - ETA: 16s - loss: 2.0348 - accuracy: 0.2991

In [39]:
# Split the data into training and testing sets (80% for training, 20% for testing)
split = int(0.8 * len(X))
x_train, x_test = X[:split], X[split:]
y_train, y_test = Y[:split], Y[split:]

In [40]:
# Compile the model again with the categorical crossentropy loss
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
# Train the model using the train-test split
model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test))


Epoch 1/15
19/19 [==============================] - 20s 1s/step - loss: 0.1511 - accuracy: 0.9576 - val_loss: 1.3101 - val_accuracy: 0.7095
Epoch 2/15
19/19 [==============================] - 18s 920ms/step - loss: 0.0754 - accuracy: 0.9796 - val_loss: 1.1694 - val_accuracy: 0.7500
Epoch 3/15
19/19 [==============================] - 17s 910ms/step - loss: 0.0439 - accuracy: 0.9932 - val_loss: 1.1942 - val_accuracy: 0.7635
Epoch 4/15
19/19 [==============================] - 17s 917ms/step - loss: 0.0249 - accuracy: 0.9966 - val_loss: 1.2824 - val_accuracy: 0.7230
Epoch 5/15
19/19 [==============================] - 17s 906ms/step - loss: 0.0141 - accuracy: 0.9983 - val_loss: 1.3783 - val_accuracy: 0.7230
Epoch 6/15
19/19 [==============================] - 17s 910ms/step - loss: 0.0087 - accuracy: 1.0000 - val_loss: 1.3486 - val_accuracy: 0.7500
Epoch 7/15
19/19 [==============================] - 20s 1s/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 1.3684 - val_accuracy: 0.7365
Epoch

In [42]:
# Make predictions on the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)


5/5 [==============================] - 2s 279ms/step


In [43]:
# Generate classification report and confusion matrix
report = classification_report(y_true_classes, y_pred_classes, target_names=CATEGORIES)
print("Classification Report:")
print(report)

Classification Report:
               precision    recall  f1-score   support

        tulip       0.82      0.64      0.72        14
      peonies       0.64      0.64      0.64        14
      orchids       0.87      0.72      0.79        18
       lilies       0.60      0.75      0.67        12
   hydrangeas       0.69      0.75      0.72        12
     hibiscus       0.63      0.80      0.71        15
    gardenias       0.83      0.88      0.86        17
 garden_roses       0.53      0.59      0.56        17
      daisies       1.00      0.73      0.85        15
bougainvillea       1.00      0.93      0.96        14

     accuracy                           0.74       148
    macro avg       0.76      0.74      0.75       148
 weighted avg       0.76      0.74      0.75       148



In [44]:
con_m = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:")
print(con_m)

Confusion Matrix:
[[ 9  0  0  1  0  1  0  3  0  0]
 [ 0  9  0  0  1  2  0  2  0  0]
 [ 0  1 13  2  0  0  0  2  0  0]
 [ 0  0  0  9  1  1  1  0  0  0]
 [ 0  0  0  1  9  1  0  1  0  0]
 [ 1  1  0  1  0 12  0  0  0  0]
 [ 0  1  0  0  1  0 15  0  0  0]
 [ 0  2  2  0  0  2  1 10  0  0]
 [ 0  0  0  1  1  0  1  1 11  0]
 [ 1  0  0  0  0  0  0  0  0 13]]


In [48]:
# Perform a prediction on an individual image
img_path = '/content/drive/MyDrive/flowers/garden_roses/garden_roses_00065.jpg'
img = cv2.imread(img_path)
img = cv2.resize(img, (100, 100))
img = img / 255.0
img = np.expand_dims(img, axis=0)
prediction = model.predict(img)
class_index = np.argmax(prediction)
class_label = CATEGORIES[class_index]
accuracy = prediction[0][class_index] * 100

print("Prediction:", class_label, "|| Accuracy:", accuracy, "%")

1/1 [==============================] - 0s 26ms/step
Prediction: garden_roses || Accuracy: 99.97768998146057 %
